# Generate pairs of images for mat and camera frames.

In [3]:
import os
import datetime
from zoneinfo import ZoneInfo
import pandas as pd
import numpy as np
import ffmpeg
import PIL
from PIL import Image
from PIL import ImageOps
# from PIL import Resampling
from matplotlib import cm

Magic number for image alignment.

In [4]:
camera_image_width, camera_image_height = 80, 60
# camera_image_rescale = 1.5

mat_image_width, mat_image_height = 27, 64
# camera_min_value, camera_max_value = 7886.0, 8171.0

Where to find the files we're after for this data fusion, and where to write the results.

In [5]:
camera_path = os.path.join('data', 'camera_data') # Infra Red camera data'
mat_path = os.path.join('data', 'mat_data') # ..\Pressure Mat Data'

# centre_camera_path = os.path.join(camera_path, 'tests', 'subject1', 'Centre 1')
# centre_mat_path = os.path.join(mat_path, 'tests', 'subject1', 'Centre 1')

In [6]:
this_camera_path = os.path.join(camera_path, 'Lyne', 'ADL IR', 'LIRADL2')
this_mat_path = os.path.join(mat_path, 'Lyne', 'ADL Mat', 'LMATADL2')

mat_csv_file = 'LA2.csv'

outpath = os.path.join('data', 'paired', 'lald2')

# camera_frame_number = 2
# mat_frame_number = 700

In [7]:
this_camera_path = os.path.join(camera_path, 'Pablo', 'Falls IR', 'PDRIRF33')
this_mat_path = os.path.join(mat_path, 'Pablo', 'Falls PM', 'PDRPMF06')

mat_csv_file = '6p.csv'

outpath = os.path.join('data', 'paired', 'p_fall_06')

# camera_frame_number = 2
# mat_frame_number = 700

Read the timestamp of a camera file.

In [8]:
def camera_date_reader(d):
    try:
        return datetime.datetime.strptime(d, 'frame_%Y-%m-%dT%H_%M_%S.%f.csv')
    except ValueError:
        return datetime.datetime.strptime(d, 'frame_%Y-%m-%dT%H_%M_%S.csv')

Read the timestamp in a mat file.

In [32]:
def mat_date_reader(d):
    return datetime.datetime.fromisoformat(d)
    # try:
    #     return datetime.datetime.strptime(d, '%Y-%m-%dT%H:%M:%S.%f%z')
    # except ValueError:
    #     return datetime.datetime.strptime(d, '%Y-%m-%dT%H:%M:%S%z')

Read all the camera files, store them in order (this follows the timestamps)

In [10]:
def find_camera_files(path):
    camera_files = {}
    for f in os.listdir(path):
        fp = os.path.join(path, f)
        if os.path.isfile(fp) and 'ContDrc' not in f:
            tm = camera_date_reader(f).replace(tzinfo=ZoneInfo('Europe/London'))
            camera_files[tm] = fp
            # print(f, tm)
    return {t: camera_files[t] for t in sorted(camera_files)}

In [11]:
# camera_files = find_camera_files(this_camera_path)
# camera_files

Load the camera data.

Open each file in turn, read it (with `loadtxt`) into a Numpy array, reshape that array into the 60×80 grid of the image, shove the array into a list.

When done, use `stack` to smash the list together into one big 3d array. First index is time, second is short side, third is long side of image.

In [12]:
def load_all_camera_data(file_times):
    camera_frames = []
    for f in file_times:
        arr = np.loadtxt(file_times[f], delimiter=',', usecols=list(range(80)), max_rows=60).reshape((1, 60, 80))
        camera_frames.append(arr)
    return np.vstack(camera_frames)

In [13]:
# camera_frames = load_all_camera_data(camera_files)
# camera_frames.shape

Normalise all the values of the camera data, so they go from 0 to 1.

In [14]:
# camera_min_value, camera_max_value = np.min(camera_frames), np.max(camera_frames)
# camera_frames = (camera_frames - cmin) / (cmax - cmin)
# camera_frames

In [15]:
# mframe = pd.read_csv(os.path.join(this_mat_path,mat_csv_file), 
#                      converters={'Timestamp': mat_date_reader},
#                     index_col='Frame')
# mframe.dtypes

In [16]:
# mframe

Capture the mat and camera times

In [17]:
# mat_times = [t.to_pydatetime() for t in mframe['Timestamp']]
# mat_times

In [18]:
# camera_times = list(sorted(camera_files.keys()))
# camera_times

Sanity check: do the times cover a sensible period (on one occasion, there were a couple of camera frames that were about three days later than the others. That led to a long video being created!)

In [19]:
# start_time = max(mat_times[0], camera_times[0])
# end_time = min(mat_times[-1], camera_times[-1])
# start_time, end_time

Build the structure of the overlaid video.

Go from the `start_time` to the `end_time`, in steps of `frame_delta`. At each step, find the most recent mat or camera time that's less than the `frame_time` and add it to `frame_sources`. Then advance `frame_time`.

You're left with a `dict` of times of frames in the composite video. For each entry in the `dict`, you can find the `mat_time` and `camera_time` to extract the data for that frame of the composite video.

In [20]:
def find_timestamp_pairs(camera_times, mat_times, start_time, end_time):
    valid_frame_times = [t for t in camera_times if t >= start_time and t <= end_time]
    
    pairs = []
    for frame_time in valid_frame_times:
        closest_mat_frame = mat_times[0]
        closest_delta = abs(closest_mat_frame - frame_time)
        for t in mat_times:
            d = abs(t - frame_time)
            if d < closest_delta:
                closest_delta = d
                closest_mat_frame = t
        pairs += [{'camera_time': frame_time, 'mat_time': closest_mat_frame}]
    return pairs

A couple of functions that convert a camera frame array to an image, and a mat frame array to an image.

In [21]:
def camera_image_for_timestamp(timestamp, camera_files, camera_min_value, camera_max_value):
    if timestamp is None:
        return Image.new(mode='L', size=(camera_image_width, camera_image_height), color=0)
    else:
        f = camera_files[timestamp]
        arr = np.loadtxt(f, delimiter=',', usecols=list(range(80)), max_rows=60).reshape((camera_image_height, camera_image_width))
        arr = (arr - camera_min_value) / (camera_max_value - camera_min_value)
        arr = np.maximum(arr, 0.0)
        arr = np.minimum(arr, 1.0)
        im = Image.fromarray((arr * 255).astype(np.uint8), mode='L')
        # im = im.convert(mode='L')
        return im

In [22]:
def mat_image_for_timestamp(timestamp, mat_data):
    if timestamp is None:
        return Image.new(mode='L', size=(int(camera_image_width * camera_image_rescale), int(camera_image_height * camera_image_rescale)), color=0)
    else:
        mat_line = mat_data.loc[mat_data['Timestamp'] == timestamp]
        mat_line = mat_line.drop(['Timestamp', 'Range Min (mmHg)', 'Range Max (mmHg)'], axis='columns')
        arr = (mat_line.to_numpy().astype(np.float64) / 104).reshape(mat_image_height, mat_image_width)
        im = Image.fromarray(np.rot90(arr * 255, 3).astype(np.uint8), mode='L')
        im = ImageOps.flip(im)
        return im

In [21]:
# p = pairs[60]
# p

In [22]:
# camera_image_for_timestamp(p['camera_time'], camera_min_value, camera_max_value).save('sample_camera.png')

In [23]:
# mat_image_for_timestamp(p['mat_time']).save('sample_mat.png')

In [24]:
# os.makedirs(outpath, exist_ok=True)

In [25]:
# with open(os.path.join(outpath, 'index.txt'), 'w') as f:
#     print('index,camera_time,mat_time', file=f)
#     for i, p in enumerate(pairs):
#         camera_image_for_timestamp(p['camera_time'], camera_min_value, camera_max_value).save(os.path.join(outpath, f'{i:03}.ir.png'))
#         mat_image_for_timestamp(p['mat_time']).save(os.path.join(outpath, f'{i:03}.pm.png'))
#         print(f"{i:03},{p['camera_time'].isoformat()},{p['mat_time'].isoformat()}", file=f)

In [26]:
# p

In [23]:
def pair_one_activity(camera_path, mat_path, mat_csv_file, out_path):
    camera_files = find_camera_files(camera_path)
    camera_data = load_all_camera_data(camera_files)
    camera_min_value, camera_max_value = np.min(camera_data), np.max(camera_data)
    camera_times = list(sorted(camera_files.keys()))
    mat_data = pd.read_csv(os.path.join(mat_path,mat_csv_file), 
                     converters={'Timestamp': mat_date_reader},
                    index_col='Frame')
    mat_times = [t.to_pydatetime() for t in mat_data['Timestamp']]
    start_time = max(mat_times[0], camera_times[0])
    end_time = min(mat_times[-1], camera_times[-1])
    print(start_time, end_time)
    time_pairs = find_timestamp_pairs(camera_times, mat_times, start_time, end_time)
    os.makedirs(out_path, exist_ok=True)
    with open(os.path.join(out_path, 'index.txt'), 'w') as f:
        print('index,camera_time,mat_time', file=f)
        for i, p in enumerate(time_pairs):
            camera_image_for_timestamp(p['camera_time'], camera_files, camera_min_value, camera_max_value).save(os.path.join(out_path, f'{i:03}.ir.png'))
            mat_image_for_timestamp(p['mat_time'], mat_data).save(os.path.join(out_path, f'{i:03}.pm.png'))
            print(f"{i:03},{p['camera_time'].isoformat()},{p['mat_time'].isoformat()}", file=f)

In [28]:
pair_one_activity(this_camera_path, this_mat_path, mat_csv_file, outpath)

2024-07-09 10:38:29.104000+01:00 2024-07-09 10:38:35.072000+01:00


In [42]:
for root, dirs, files in os.walk(os.path.join('data', 'trimmed_csv')):
    for file in files:
        if file.endswith('.csv'):
            if 'Fall' in root or 'fall' in root:
                out_dir = os.path.join('data', 'mat_images', 'falls')
            else:
                out_dir = os.path.join('data', 'mat_images', 'adl')
            try:
                os.makedirs(out_dir)
            except FileExistsError:
                pass
            out_file_name = root.replace(' ', '_')
            out_file_name = out_file_name.replace('/', '_')
            
            # out_file = os.path.join(out_dir, out_file_name)

            mat_data = pd.read_csv(os.path.join(root, file), 
                     converters={'Timestamp': mat_date_reader},
                    index_col='Frame')
            mat_arr = mat_data.drop(['Timestamp', 'Range Min (mmHg)', 'Range Max (mmHg)'], axis='columns').to_numpy().astype(np.float64) / 104

            for i, mat_row in enumerate(mat_arr):
                # print(out_file_name, i, np.count_nonzero(mat_row >= 0.1))
                if np.count_nonzero(mat_row >= 0.1) >= 10:
                    img_arr = mat_row.reshape(mat_image_height, mat_image_width)
                    im = Image.fromarray(np.rot90(img_arr * 255, 3).astype(np.uint8), mode='L')
                    im = ImageOps.flip(im)
                    im.save(os.path.join(out_dir, f'{out_file_name}_{i:03}.png'))
            # print(root, file, out_file) # uncomment this to see what files are being trimmed


In [38]:
np.count_nonzero(mat_row >= 10)

0